In [35]:
!nvidia-smi


Fri Sep 13 13:37:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Compilação com NVCC


In [36]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Overwriting exemplo1.cu


In [37]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1

In [38]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


### Transformações e Operações Matemáticas com Vetores na GPU


In [39]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Overwriting exemplo2.cu


In [40]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [41]:
!./exemplo2


V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


### Leitura de arquivos pela GPU


In [42]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}


Overwriting stocks.cu


In [43]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [44]:
!./stocks < stocks-google.txt


Tempo de LEITURA (em segundos)  0.000575256
Tempo de CÓPIA (em segundos)  0.224632


## Exercício 1: Leitura e Transferência de Dados

In [59]:
%%writefile exercicio1.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>
#include <chrono>
#include <fstream>  // Para salvar os dados lidos
using namespace std;

int main() {
    int n = 2518;  // Ajuste o número de elementos conforme o arquivo real
    double value = 0.0;
    std::chrono::duration<double> diff;

    // Medir o tempo de leitura dos dados no host
    auto leitura_i = std::chrono::steady_clock::now();

    // Criar um vetor na CPU (host_vector) e ler os valores do arquivo
    thrust::host_vector<double> host(n, 0);
    for (int i = 0; i < n; i++) {
        cin >> value;  // Ler o valor da entrada padrão (ou arquivo)
        host[i] = value;
    }

    auto leitura_f = std::chrono::steady_clock::now();
    diff = leitura_f - leitura_i;
    cout << "Tempo de LEITURA (em segundos): " << diff.count() << endl;

    // Salvar os dados lidos para uso posterior
    ofstream outfile("dados.bin", ios::binary);
    outfile.write(reinterpret_cast<char*>(host.data()), n * sizeof(double));
    outfile.close();

    // Medir o tempo de cópia dos dados para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Criar um vetor na GPU (device_vector) e copiar os dados do host para ele
    thrust::device_vector<double> dev(host);

    auto copia_f = std::chrono::steady_clock::now();
    diff = copia_f - copia_i;
    cout << "Tempo de CÓPIA para a GPU (em segundos): " << diff.count() << endl;

    return 0;
}


Overwriting exercicio1.cu


In [60]:
!nvcc -arch=sm_75 -std=c++14 exercicio1.cu -o exercicio1

In [61]:
!./exercicio1 < stocks-google.txt

Tempo de LEITURA (em segundos): 0.00055173
Tempo de CÓPIA para a GPU (em segundos): 0.189592


### Exercício 2: Cálculo de Médias e Extremos



In [71]:
%%writefile exercicio2.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/reduce.h>
#include <thrust/extrema.h>
#include <iostream>
#include <fstream>

int main() {
    thrust::host_vector<float> h_stock_prices;

    // Abrir o arquivo stocks-google.txt
    std::ifstream file("/content/stocks-google.txt");
    if (file.is_open()) {
        float price;
        // Ler os preços do arquivo e armazenar no host_vector
        while (file >> price) {
            h_stock_prices.push_back(price);
        }
        file.close();
    }

    // Transferir os dados para a GPU (device_vector)
    thrust::device_vector<float> d_stock_prices = h_stock_prices;

    // Cálculo da média do período total (últimos 10 anos)
    float sum_total = thrust::reduce(d_stock_prices.begin(), d_stock_prices.end(), 0.0f);
    float average_total = sum_total / d_stock_prices.size();

    // Cálculo da média dos últimos 365 dias
    int days_in_year = 365;
    int start_index = d_stock_prices.size() - days_in_year;  // Índice de início dos últimos 365 dias
    if (start_index < 0) start_index = 0;  // Certificar que não acessamos fora do vetor

    float sum_last_year = thrust::reduce(d_stock_prices.begin() + start_index, d_stock_prices.end(), 0.0f);
    float average_last_year = sum_last_year / (d_stock_prices.end() - d_stock_prices.begin() - start_index);

    // Encontrar o maior e o menor preço no período total
    auto minmax_total = thrust::minmax_element(d_stock_prices.begin(), d_stock_prices.end());
    float min_total = *minmax_total.first;
    float max_total = *minmax_total.second;

    // Encontrar o maior e o menor preço nos últimos 365 dias
    auto minmax_last_year = thrust::minmax_element(d_stock_prices.begin() + start_index, d_stock_prices.end());
    float min_last_year = *minmax_last_year.first;
    float max_last_year = *minmax_last_year.second;

    // Exibir os resultados
    std::cout << "Preço médio total: " << average_total << std::endl;
    std::cout << "Preço médio nos últimos 365 dias: " << average_last_year << std::endl;
    std::cout << "Maior preço total: " << max_total << std::endl;
    std::cout << "Menor preço total: " << min_total << std::endl;
    std::cout << "Maior preço nos últimos 365 dias: " << max_last_year << std::endl;
    std::cout << "Menor preço nos últimos 365 dias: " << min_last_year << std::endl;

    return 0;
}


Overwriting exercicio2.cu


In [72]:
!nvcc -arch=sm_75 -std=c++14 exercicio2.cu -o exercicio2


In [73]:
!./exercicio2

Preço médio total: 1580.08
Preço médio nos últimos 365 dias: 1615.81
Maior preço total: 2200
Menor preço total: 1015.24
Maior preço nos últimos 365 dias: 2200
Menor preço nos últimos 365 dias: 1060.5


### Exercício 3: Transformações entre Vetores

In [68]:
%%writefile exercicio3.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>    // Para transformações ponto a ponto
#include <thrust/reduce.h>       // Para calcular a média
#include <iostream>
#include <fstream>               // Para leitura do arquivo

int main() {
    int n = 3520;  // Total de registros (dias de negociação)
    double aapl_price, msft_price;

    // Criar vetores para armazenar os preços das ações da Apple e Microsoft
    thrust::host_vector<double> aapl_host(n, 0);
    thrust::host_vector<double> msft_host(n, 0);

    // Ler os dados do arquivo stocks2.txt
    std::ifstream infile("stocks2.txt");
    for (int i = 0; i < n; i++) {
        char comma;  // Para ignorar a vírgula
        infile >> aapl_price >> comma >> msft_price;
        aapl_host[i] = aapl_price;
        msft_host[i] = msft_price;
    }
    infile.close();

    // Transferir os dados para a GPU
    thrust::device_vector<double> aapl_dev = aapl_host;
    thrust::device_vector<double> msft_dev = msft_host;

    // Criar um vetor para armazenar as diferenças entre os preços
    thrust::device_vector<double> diff_dev(n);

    // Calcular a diferença ponto a ponto entre os preços da Apple e Microsoft
    thrust::transform(aapl_dev.begin(), aapl_dev.end(), msft_dev.begin(), diff_dev.begin(), thrust::minus<double>());

    // Calcular a soma das diferenças
    double soma_diferencas = thrust::reduce(diff_dev.begin(), diff_dev.end(), 0.0);

    // Calcular a média das diferenças
    double media_diferenca = soma_diferencas / n;

    // Imprimir o resultado
    std::cout << "A diferença média entre os preços das ações da Apple e Microsoft é: " << media_diferenca << std::endl;

    return 0;
}


Writing exercicio3.cu


In [69]:
!nvcc -arch=sm_75 -std=c++14 exercicio3.cu -o exercicio3


In [70]:
!./exercicio3


A diferença média entre os preços das ações da Apple e Microsoft é: -265.232


Microsoft teve um preço médio maior que a Apple por 265.23 unidades no período analisado.